In [1]:
import xml.etree.ElementTree as ET
import os
from pathlib import Path
import shutil
import random
from sklearn.model_selection import train_test_split

class DatasetConverter:
    def __init__(self, input_dir, output_dir, class_names=['full_lined']):
        self.input_dir = Path(input_dir)
        self.output_dir = Path(output_dir)
        self.class_names = class_names
        
        # Create directory structure
        self.dataset_dir = self.output_dir / 'dataset'
        for split in ['train', 'val', 'test']:
            (self.dataset_dir / split / 'images').mkdir(parents=True, exist_ok=True)
            (self.dataset_dir / split / 'labels').mkdir(parents=True, exist_ok=True)

    def convert_bbox_to_yolo(self, size, box):
        dw = 1./size[0]
        dh = 1./size[1]
        x = (box[0] + box[2])/2.0
        y = (box[1] + box[3])/2.0
        w = box[2] - box[0]
        h = box[3] - box[1]
        return (x*dw, y*dh, w*dw, h*dh)

    def convert_annotation(self, xml_path):
        tree = ET.parse(xml_path)
        root = tree.getroot()
        
        size = root.find('size')
        w = int(size.find('width').text)
        h = int(size.find('height').text)
        
        # Create temporary file in a writable directory
        temp_dir = '/kaggle/working/temp_labels'  # Kaggle's writable directory
        os.makedirs(temp_dir, exist_ok=True)
        out_file = Path(temp_dir) / f"{xml_path.stem}.txt"
        
        with open(out_file, 'w') as f:
            for obj in root.iter('object'):
                cls_name = obj.find('name').text
                # if cls_name not in self.class_names:
                #     continue
                # cls_id = self.class_names.index(cls_name)
                
                xmlbox = obj.find('bndbox')
                b = (float(xmlbox.find('xmin').text),
                     float(xmlbox.find('ymin').text),
                     float(xmlbox.find('xmax').text),
                     float(xmlbox.find('ymax').text))

                bb = self.convert_bbox_to_yolo((w,h), b)
                f.write(f"0 {bb[0]:.6f} {bb[1]:.6f} {bb[2]:.6f} {bb[3]:.6f}\n")
        
        return out_file

    def create_yaml(self):
        yaml_content = f"""path: {self.dataset_dir.absolute()}
train: train/images
val: val/images
test: test/images

nc: {len(self.class_names)}
names: {self.class_names}"""

        with open(self.output_dir / 'dataset.yaml', 'w') as f:
            f.write(yaml_content)

    def process_dataset(self, train_size=0.7, val_size=0.2, test_size=0.1):
        # Get all XML files
        xml_files = list(self.input_dir.glob('*.xml'))
        
        # Split dataset
        train_val_files, test_files = train_test_split(xml_files, test_size=test_size, random_state=42)
        train_files, val_files = train_test_split(train_val_files, 
                                                test_size=val_size/(train_size + val_size), 
                                                random_state=42)
        
        # Process each split
        splits = {
            'train': train_files,
            'val': val_files,
            'test': test_files
        }
        
        for split_name, files in splits.items():
            print(f"Processing {split_name} split...")
            for xml_file in files:
                try:
                    # Convert annotation
                    txt_file = self.convert_annotation(xml_file)
                    
                    # Get corresponding image file
                    img_file = xml_file.with_suffix('.jpg')
                    if not img_file.exists():
                        img_file = xml_file.with_suffix('.jpeg')
                    if not img_file.exists():
                        # print(f"Warning: No image file found for {xml_file}")
                        continue
                    # Copy files to appropriate directories
                    shutil.copy2(img_file, 
                               self.dataset_dir / split_name / 'images' / img_file.name)
                    shutil.copy2(txt_file, 
                               self.dataset_dir / split_name / 'labels' / txt_file.name)
                except:
                    pass
        
        # Create YAML file
        self.create_yaml()
        print("Dataset preparation completed!")

In [2]:
input_directory = "/kaggle/input/tncr-table-detection/verified"  # folder containing .jpg and .xml files
output_directory = "/kaggle/working/"  # where to save the processed dataset
class_names = [
    'full_lined',
    'merged_cells',
    'no_lines',
    'partial_lined',
    'partial_lined_merged_cells'
]  # list of your class names

converter = DatasetConverter(input_directory, output_directory, class_names)
converter.process_dataset(train_size=0.7, val_size=0.2, test_size=0.1)

Processing train split...
Processing val split...
Processing test split...
Dataset preparation completed!


In [3]:
!pip install -q ultralytics
!pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.1 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.9
    Uninstalling widgetsnbextension-3.6.9:
      Successfully uninstalled widgetsnbextension-3.6.9
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.11
    Uninstalling jupyterlab_widgets-3.0.11:
      Successfully uninstalled jupyterlab_widgets-3.0.11
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 

In [4]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11s.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="/kaggle/working/dataset.yaml", epochs=100, imgsz=640)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 18.4M/18.4M [00:00<00:00, 159MB/s]


Ultralytics 8.3.39 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/kaggle/working/dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tr

100%|██████████| 755k/755k [00:00<00:00, 17.2MB/s]


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics

...

    100/100      4.33G     0.1535     0.1447     0.7992         22        640: 100%|██████████| 169/169 [00:46<00:00,  3.66it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:05<00:00,  4.26it/s]

                   all        782       1167      0.982      0.984      0.994       0.98



100 epochs completed in 1.484 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 19.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 238 layers, 9,414,735 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:07<00:00,  3.57it/s]


                   all        782       1167      0.982      0.981      0.994       0.98
            full_lined        782       1167      0.982      0.981      0.994       0.98
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/train


In [5]:
!rm -rf "/kaggle/working/dataset"
!rm -rf "/kaggle/working/temp_labels"

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
